In [58]:
from importlib import reload
import logging
import taxbalance

reload(logging)
reload(taxbalance)

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s:%(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')


balance = taxbalance.UFNSTaxBalance().add_from_excel("data/Выписка операций по расчету с бюджетом Ангарский (2020).xlsx")\
    .add_from_excel("data/Выписка операций по расчету с бюджетом Ангарский (2022).xlsx")\
    .add_from_excel("data/Выписка операций по расчету с бюджетом Ангарский (2021).xlsx")



2023-04-04 09:35:04 INFO:Успешно импортировано 816 строк.
2023-04-04 09:35:04 INFO:Загрузка файла завершена. В балансе содержится 816 операций.
2023-04-04 09:35:07 INFO:Успешно импортировано 913 строк.
2023-04-04 09:35:07 INFO:Загрузка файла завершена. В балансе содержится 1729 операций.
2023-04-04 09:35:09 INFO:Успешно импортировано 764 строк.
2023-04-04 09:35:09 INFO:Загрузка файла завершена. В балансе содержится 2493 операций.


In [104]:
# Импорт Банковских платежей
reload(logging)
reload(taxbalance)
logging.basicConfig(level=logging.INFO)
sber = taxbalance.SberPayments()\
    .add_from_excel('/Users/maksimtunev/PycharmProjects/balanceTax/TaxBalance/data/Ангарский июль-декабрь 2020.xlsx')\
    .add_from_excel('/Users/maksimtunev/PycharmProjects/balanceTax/TaxBalance/data/СберБизнес. Выписка за 2021.01.01-2021.12.31 счёт 40702810465000000826.xlsx')\
    .add_from_excel('/Users/maksimtunev/PycharmProjects/balanceTax/TaxBalance/data/СберБизнес. Выписка за 2022.01.01-2022.12.31 счёт 40702810465000000826.xlsx')


/Users/maksimtunev/PycharmProjects/balanceTax/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
INFO:root:Успешно импортировано 65 строк.
INFO:root:Загрузка файла завершена. В платежах содержится 65 операций.
INFO:root:Успешно импортировано 261 строк.
INFO:root:Загрузка файла завершена. В платежах содержится 326 операций.
/Users/maksimtunev/PycharmProjects/balanceTax/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
INFO:root:Успешно импортировано 311 строк.
INFO:root:Загрузка файла завершена. В платежах содержится 637 операций.


In [85]:
sber._operations

,operation_date,operation_sum,document_number,reason,decision_number
0,2020-12-18,4569.08,2179,По решению о взыскании № 685 от 11.12.2020 на ...,685
1,2020-12-18,8711.62,2177,По решению о взыскании № 685 от 11.12.2020 на ...,685
2,2020-12-18,15558.15,2179,По решению о взыскании № 685 от 11.12.2020 на ...,685
3,2020-12-18,122143.06,2179,По решению о взыскании № 685 от 11.12.2020 на ...,685
4,2020-12-19,30006.15,2179,По решению о взыскании № 685 от 11.12.2020 на ...,685
...,...,...,...,...,...
633,2022-11-24,5509.11,12790,По решению о взыскании № 2820 от 29.03.2022 на...,2820
631,2022-11-24,216.64,12790,По решению о взыскании № 2820 от 29.03.2022 на...,2820
634,2022-11-24,39531.47,12790,По решению о взыскании № 2820 от 29.03.2022 на...,2820
635,2022-11-25,3643.84,12790,По решению о взыскании № 2820 от 29.03.2022 на...,2820


In [94]:
sber._operations.groupby(by=['decision_number', 'document_number', ]).sum(numeric_only=True)


operation_sum
decision_number document_number               
1061            3188                    492.03
                3189                 192954.00
1133            4797                  19086.16
                4798                   2500.83
                4799                   4424.52
...                                        ...
8785            35659                 21567.00
                35660                 25797.47
                35661                  3298.69
                35662                  5980.32
                35663                   129.05

[108 rows x 1 columns]

In [101]:
# Проверяем может ли к одному платежному документу относится два решения

payment_docs = sber._operations.document_number.unique()

for doc in payment_docs:
    unique_list = sber._operations[sber._operations['document_number'] == doc]['decision_number'].unique()
    decision_count = len(unique_list)
    if decision_count > 1:
        logging.warning(f'Найдена проблема: документ №{doc} содержит {decision_count} решений')
    else:
        logging.warning(f'Успех: документ №{doc} содержит {decision_count} решений: {unique_list}')